In [ ]:
import numpy as np
import time
import pickle
import sys
import os
import csv

from RhoAndBeta import CalcRhoAndBetaVectors
from UtilitiesOptimization import SubgrAlgSavPrimDualObjInd, \
    SubgrAlgSavPrimDualObjFn_L2Ind
from SimulationCode import RunOnlyGreedy, ExpPareto

In [ ]:
## Read the data created in CreateDataJupNot
suffix='InstanceInfo/Ins1LongRun/'
num_impressions=pickle.load(open(suffix+'num_impressions'+'.p',"rb"))
numCampaigns=pickle.load(open(suffix+'numCampaigns'+'.p',"rb"))
num_edges=pickle.load(open(suffix+'num_edges'+'.p',"rb"))
index_Imps=pickle.load(open(suffix+'index_Imps'+'.p',"rb"))
index_sizeCamps=pickle.load(open(suffix+'index_sizeCamps'+'.p',"rb"))
index_startCamp=pickle.load(open(suffix+'index_startCamp'+'.p',"rb"))
vector_m=pickle.load(open(suffix+'vector_m'+'.p',"rb"))
vector_maxbid=pickle.load(open(suffix+'vector_maxbid'+'.p',"rb"))
vector_r=pickle.load(open(suffix+'vector_r'+'.p',"rb"))
vector_s=(pickle.load(open(suffix+'vector_s'+'.p',"rb"))).astype(int)
ext_s=pickle.load(open(suffix+'ext_s'+'.p',"rb"))
adverPerImp=(pickle.load(open(suffix+'adverPerImp'+'.p',"rb"))).astype(int)
UB_bidsPerImp = pickle.load(open(suffix+'UB_bids'+'.p',"rb"))
vector_ctr=pickle.load(open(suffix+'vector_ctr'+'.p',"rb"))
vector_rctr=pickle.load(open(suffix+'vector_rctr'+'.p',"rb"))
probImp=pickle.load(open(suffix+'probImp'+'.p',"rb"))

In [ ]:
## If this parameter is true then first price auctions (\beta_i(b) =b) are used, otherwise second price.
firstPrice = True
## If this parameter is true a parameterSearch will be performd and .csv will be made,
parameterSearch = False
## We use $\tau_k = 1/m_k$
expForTau=-1.0
tau = np.power(vector_m,expForTau)
UB_bids = UB_bidsPerImp[index_Imps]

suffix='ResultsPareto/'
current_directory = os.getcwd()
results_directory = os.path.join(current_directory, suffix)
if not os.path.exists(results_directory):
    os.makedirs(results_directory)

# Parameter Search

This parameter search looks for a constant $C$, such that the dual method shows 
empirical convergence. Given that here we only look for training convergence, we don't 
need to cross-validate or similar.

In [ ]:
parameterSearch = False

In [ ]:
if parameterSearch:
    input_var = input("This will execute a simple parameter search.\
    If you just wanted to run just a full long run do parameterSearch= False  \
    and kill this execution. To continue press enter.")
    c=np.array([1.0,0.1,0.01,0.001,0.0001,0.00001,0.000001,0.0000001])
    p_grad_Type = 0
    num_it = 1000
    it_per_cal = 250
    init_lam=np.zeros((numCampaigns))
    alphas_pure=np.fromfunction(lambda i, j: (1/(np.sqrt(i + 1))), \
        (num_it, 1), dtype=int)[:,0] 
    
#     dualObjFn, primalObjFn, dualObjFnAvg, primalObjFnAvg, budget_used, \
#     budget_LamAvgUse, dual_vars, dual_varsAvg = [], [], [], [], [], [], [], []

    nameResults='ParameterSearch'+'It_'+str(num_it)
    f = open(suffix+nameResults+'.csv', 'wt')
    writer = csv.writer(f, lineterminator='\n')
    writer.writerow( ('Auction Type', 'Problem Type', 'Ite', 'Cte', 'DualFnValue',\
        'PrimalValue','DualFnValueAvg','PrimalFnValueAvg') )
    
    print('Using First Price Auctions')
    firstPrice = True
    for c_val in c:
        print('c_val: ',c_val, end =', Methods: ')
        print('Indicator Case', end =' ,')
        p_grad_Type = 0
        alphas=c_val*alphas_pure
        [dual_FnValues,primal_GivenMu,budget_used,dual_vars,dual_AvgLamFnValues,\
            primal_AvgLamGivenMu,budget_LamAvgUse,dual_varsAvg]= SubgrAlgSavPrimDualObjInd(\
            init_lam, num_it, alphas, vector_r, vector_ctr, vector_rctr, vector_s, ext_s, \
            vector_m, num_impressions, numCampaigns, num_edges, index_sizeCamps, index_Imps,\
            UB_bids, firstPrice, adverPerImp, it_per_cal, p_grad_Type)
        numSaved=len(budget_used)
        for t in range(numSaved):
            writer.writerow(('First price', 'Indicator', (t+1)*it_per_cal,c_val,dual_FnValues[t],\
                primal_GivenMu[t],dual_AvgLamFnValues[t],primal_AvgLamGivenMu[t]))
            
        print('L2 penalization wout indicator', end =' ,')
        p_grad_Type = 1
        alphas=c_val*alphas_pure
        [dual_FnValues,primal_GivenMu,budget_used,dual_vars,dual_AvgLamFnValues,\
            primal_AvgLamGivenMu,budget_LamAvgUse,dual_varsAvg]= SubgrAlgSavPrimDualObjFn_L2Ind(\
            init_lam, num_it, alphas, vector_r, vector_ctr, vector_rctr, vector_s, ext_s, vector_m,\
            num_impressions, numCampaigns, num_edges, index_sizeCamps, index_Imps,  UB_bids, firstPrice,\
            adverPerImp, it_per_cal, p_grad_Type, tau, False)
        numSaved=len(budget_used)
        for t in range(numSaved):
            writer.writerow(('First price', 'L2 Wout Ind', (t+1)*it_per_cal,c_val,dual_FnValues[t],\
                primal_GivenMu[t],dual_AvgLamFnValues[t],primal_AvgLamGivenMu[t]))
            
        print('L2 with indicator')
        p_grad_Type = 2
        alphas=c_val*alphas_pure
        [dual_FnValues,primal_GivenMu,budget_used,dual_vars,dual_AvgLamFnValues,\
            primal_AvgLamGivenMu,budget_LamAvgUse,dual_varsAvg]= SubgrAlgSavPrimDualObjFn_L2Ind(\
            init_lam, num_it, alphas, vector_r, vector_ctr, vector_rctr, vector_s, ext_s, vector_m,\
            num_impressions, numCampaigns, num_edges, index_sizeCamps, index_Imps,  UB_bids, firstPrice,\
            adverPerImp, it_per_cal, p_grad_Type, tau, True)
        numSaved=len(budget_used)
        for t in range(numSaved):
            writer.writerow(('First price', 'L2 + Indicator', (t+1)*it_per_cal,c_val,dual_FnValues[t],\
                primal_GivenMu[t],dual_AvgLamFnValues[t],primal_AvgLamGivenMu[t]))

            
    print('Using Second Price Auctions')
    firstPrice = False
    for c_val in c:
        print('c_val: ',c_val, end =', Methods: ')
        print('Indicator Case', end =' ,')
        p_grad_Type = 0
        alphas=c_val*alphas_pure
        [dual_FnValues,primal_GivenMu,budget_used,dual_vars,dual_AvgLamFnValues,\
            primal_AvgLamGivenMu,budget_LamAvgUse,dual_varsAvg]= SubgrAlgSavPrimDualObjInd(\
            init_lam, num_it, alphas, vector_r, vector_ctr, vector_rctr, vector_s, ext_s, \
            vector_m, num_impressions, numCampaigns, num_edges, index_sizeCamps, index_Imps,\
            UB_bids, firstPrice, adverPerImp, it_per_cal, p_grad_Type)
        numSaved=len(budget_used)
        for t in range(numSaved):
            writer.writerow(('Second price', 'Indicator', (t+1)*it_per_cal,c_val,dual_FnValues[t],\
                primal_GivenMu[t],dual_AvgLamFnValues[t],primal_AvgLamGivenMu[t]))
            
        print('L2 penalization wout indicator', end =' ,')
        p_grad_Type = 1
        alphas=c_val*alphas_pure
        [dual_FnValues,primal_GivenMu,budget_used,dual_vars,dual_AvgLamFnValues,\
            primal_AvgLamGivenMu,budget_LamAvgUse,dual_varsAvg]= SubgrAlgSavPrimDualObjFn_L2Ind(\
            init_lam, num_it, alphas, vector_r, vector_ctr, vector_rctr, vector_s, ext_s, vector_m,\
            num_impressions, numCampaigns, num_edges, index_sizeCamps, index_Imps,  UB_bids, firstPrice,\
            adverPerImp, it_per_cal, p_grad_Type, tau, False)
        numSaved=len(budget_used)
        for t in range(numSaved):
            writer.writerow(('Second price', 'L2 Wout Ind', (t+1)*it_per_cal,c_val,dual_FnValues[t],\
                primal_GivenMu[t],dual_AvgLamFnValues[t],primal_AvgLamGivenMu[t]))
            
        print('L2 with indicator')
        p_grad_Type = 2
        alphas=c_val*alphas_pure
        [dual_FnValues,primal_GivenMu,budget_used,dual_vars,dual_AvgLamFnValues,\
            primal_AvgLamGivenMu,budget_LamAvgUse,dual_varsAvg]= SubgrAlgSavPrimDualObjFn_L2Ind(\
            init_lam, num_it, alphas, vector_r, vector_ctr, vector_rctr, vector_s, ext_s, vector_m,\
            num_impressions, numCampaigns, num_edges, index_sizeCamps, index_Imps,  UB_bids, firstPrice,\
            adverPerImp, it_per_cal, p_grad_Type, tau, True)
        numSaved=len(budget_used)
        for t in range(numSaved):
            writer.writerow(('Second price', 'L2 + Indicator', (t+1)*it_per_cal,c_val,dual_FnValues[t],\
                primal_GivenMu[t],dual_AvgLamFnValues[t],primal_AvgLamGivenMu[t]))
    f.close()

# The following value are obtained just by Looking at The Parameter Search .csv

In [ ]:
## Best constants
consBestFP = [0.0001, 0.5, 0.5]
consBestSP = [0.0001, 0.5, 0.5]

# Run Pareto

In [ ]:
np.random.seed(1234)
multipliers = []
for i in range(40):
    multipliers.append(0.05+0.05*np.power(1.2,i))

num_itInd, num_itL2Ind = 5000, 5000
init_lam = np.zeros(numCampaigns)
alphas_pure = np.array([np.sqrt(1.0/(i + 1)) for i in range(num_itInd)])
alphasInd = consBestFP[0] * np.array([np.sqrt(1.0/(i + 1)) for i in range(num_itInd)])
alphasL2Ind = consBestFP[2] * np.array([np.sqrt(1.0/(i + 1)) for i in range(num_itL2Ind)])

firstPrice = True
shuffle = True
sim = 100

np.random.seed(2947)
vecOfSeeds = (np.random.choice(np.arange(100000), size = sim,\
                replace = False)).astype(int)

In [ ]:
listGreedy = RunOnlyGreedy(numCampaigns, num_impressions, num_edges, index_Imps, \
    index_sizeCamps, vector_s, vector_r, vector_m, vector_ctr, UB_bidsPerImp,\
    adverPerImp, sim, firstPrice, vecOfSeeds, shuffle)

dictToRetL2Ind = ExpPareto(numCampaigns, num_impressions, num_edges, index_Imps, \
    index_sizeCamps, vector_s, vector_r, vector_m, vector_ctr, UB_bidsPerImp,\
    adverPerImp, alphasL2Ind, num_itL2Ind, 2,\
    multipliers, init_lam, sim, firstPrice, vecOfSeeds, shuffle)

pickle.dump(listGreedy, open(results_directory+'listGreedy.p','wb'))
pickle.dump(dictToRetL2Ind, open(results_directory+'dictToRetL2Ind.p','wb'))